In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [9]:
#column = ['tweets','duygu','preprocessing']
#df = pd.read_excel("../dataset/total.xlsx")

column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")


df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [10]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [11]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,canım turkcell?? neti bir kez bile kesmedi,1
1,turkcell bana kazık atıyoo,0
2,turkcell'li olan varsa söylesin çabuk fena bi ...,1
3,ulak'ta ilk ön siparişi turkcell verdi https:/...,1
4,turkcell'in sahur interneti iyi ki var,1
...,...,...
9121,sedaya ingiltereden guzel ama yalniz kadinlar ...,0
9122,sabri reyiz ile ilgili onlarca guzel reklam ce...,0
9123,hayata gülümse diyor oç turkcell ne güzel süla...,0
9124,@myygoddemi yaşasın turkcell superonline,1


In [12]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [13]:
def tf_idf (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    tokenizer = Tokenizer(num_words=1000) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    vocabulary=[]
    for key in word_index.keys():
        vocabulary.append(key)

    pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                     ('tfid', TfidfTransformer())]).fit(X_train)

    res = dict(zip(vocabulary, pipe['tfid'].idf_))

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    
    skipped_words = 0
    embedding_dim = 1 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = res[word] #kelimenin word2vec karşılığı vektör olarak
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector 
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

    return embedding_layer,X_train,y_train,X_test,y_test

In [14]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= tf_idf (X_train,y_train,X_test,y_test)
    
    modelCNNLSTM = Sequential() 

    modelCNNLSTM.add(embedding_layer)
    modelCNNLSTM.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
    modelCNNLSTM.add(MaxPooling1D(pool_size=4))
    modelCNNLSTM.add(Dropout(0.25))
    modelCNNLSTM.add(LSTM(128))                            
    modelCNNLSTM.add(Flatten())
    modelCNNLSTM.add(Dense(1, activation='sigmoid'))
    modelCNNLSTM.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNNLSTM.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNNLSTM.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6944 - acc: 0.5131 - precision: 0.5109 - recall: 0.5313
Epoch 2/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6926 - acc: 0.5204 - precision: 0.5185 - recall: 0.5239
Epoch 3/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6911 - acc: 0.5288 - precision: 0.5270 - recall: 0.5301
Epoch 4/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6908 - acc: 0.5272 - precision: 0.5259 - recall: 0.5191 1s -
Epoch 5/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6900 - acc: 0.5301 - precision: 0.5279 - recall: 0.5393
Epoch 6/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6895 - acc: 0.5370 - precision: 0.5350 - recall: 0.5403
Epoch 7/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6887 - acc: 0.5368 - pr

257/257 [==============================] - 5s 20ms/step - loss: 0.6919 - acc: 0.5227 - precision_1: 0.5248 - recall_1: 0.5433 2s - loss: 0.6924  - ETA: 1s - loss: 0.6920 - acc: 0.5224 - precision_1: 0.5244 - recall_1: 0.54 - ETA: 1s - loss: 0.6919 - acc: 0.5232 
Epoch 4/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6907 - acc: 0.5307 - precision_1: 0.5327 - recall_1: 0.5474 0s - loss: 0.6907 - acc: 0.5287 - precis
Epoch 5/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6905 - acc: 0.5299 - precision_1: 0.5341 - recall_1: 0.5133
Epoch 6/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6902 - acc: 0.5306 - precision_1: 0.5307 - recall_1: 0.5789
Epoch 7/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6888 - acc: 0.5357 - precision_1: 0.5364 - recall_1: 0.5687
Epoch 8/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6895 - acc: 0.5339 - precision_1: 0.5363 - recall_1: 0.5428 1s - loss: 

257/257 [==============================] - 5s 18ms/step - loss: 0.6923 - acc: 0.5237 - precision_2: 0.5237 - recall_2: 0.5375 3s - loss: 0.6952 - acc: 0
Epoch 3/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6906 - acc: 0.5305 - precision_2: 0.5309 - recall_2: 0.5355
Epoch 4/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6906 - acc: 0.5316 - precision_2: 0.5326 - recall_2: 0.5270  - ETA: 0s - loss: 0.6910 - acc: 0.5306 - precision_2: 0.5319 - 
Epoch 5/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6903 - acc: 0.5338 - precision_2: 0.5334 - recall_2: 0.5501
Epoch 6/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6899 - acc: 0.5323 - precision_2: 0.5309 - recall_2: 0.5669
Epoch 7/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6891 - acc: 0.5383 - precision_2: 0.5406 - recall_2: 0.5180
Epoch 8/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6892 - acc: 0.5342 -

257/257 [==============================] - 5s 18ms/step - loss: 0.6816 - acc: 0.5580 - precision_3: 0.5594 - recall_3: 0.5202
Epoch 16/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6808 - acc: 0.5553 - precision_3: 0.5566 - recall_3: 0.5165
Epoch 17/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6804 - acc: 0.5637 - precision_3: 0.5616 - recall_3: 0.5560 
Epoch 18/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6800 - acc: 0.5624 - precision_3: 0.5645 - recall_3: 0.5219
Epoch 19/50
257/257 [==============================] - 4s 16ms/step - loss: 0.6784 - acc: 0.5608 - precision_3: 0.5591 - recall_3: 0.5496
Epoch 20/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6773 - acc: 0.5674 - precision_3: 0.5676 - recall_3: 0.5435
Epoch 21/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6771 - acc: 0.5693 - precision_3: 0.5717 - recall_3: 0.5312
Epoch 22/50
257/257 [========================

257/257 [==============================] - 4s 17ms/step - loss: 0.6888 - acc: 0.5314 - precision_4: 0.5275 - recall_4: 0.6369
Epoch 19/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6888 - acc: 0.5299 - precision_4: 0.5272 - recall_4: 0.6168
Epoch 20/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6880 - acc: 0.5318 - precision_4: 0.5274 - recall_4: 0.6495 1s - loss: 0.6866 - 
Epoch 21/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6883 - acc: 0.5405 - precision_4: 0.5375 - recall_4: 0.6066
Epoch 22/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6880 - acc: 0.5400 - precision_4: 0.5364 - recall_4: 0.6168
Epoch 23/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6869 - acc: 0.5372 - precision_4: 0.5342 - recall_4: 0.6107
Epoch 24/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6872 - acc: 0.5385 - precision_4: 0.5340 - recall_4: 0.6342 1s -
Epoch 25/50
257/257 

257/257 [==============================] - 5s 19ms/step - loss: 0.6768 - acc: 0.5693 - precision_5: 0.5688 - recall_5: 0.5797
Epoch 22/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6764 - acc: 0.5684 - precision_5: 0.5650 - recall_5: 0.6011
Epoch 23/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6769 - acc: 0.5654 - precision_5: 0.5627 - recall_5: 0.5941
Epoch 24/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6748 - acc: 0.5718 - precision_5: 0.5693 - recall_5: 0.5958
Epoch 25/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6748 - acc: 0.5691 - precision_5: 0.5691 - recall_5: 0.5758 1s - loss: 0.6745 - acc: 0.5661 - precision_5: 0.5652 - reca - ETA: 0s - loss: 0.6744 - acc: 0.5671 - precision_
Epoch 26/50
257/257 [==============================] - 5s 20ms/step - loss: 0.6739 - acc: 0.5684 - precision_5: 0.5654 - recall_5: 0.5982 0s - loss: 0.6742 - acc: 0.5679 - precision_5: 0.5637 
Epoch 27/50
257/

257/257 [==============================] - 5s 19ms/step - loss: 0.6823 - acc: 0.5536 - precision_6: 0.5562 - recall_6: 0.4826 2s
Epoch 21/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6828 - acc: 0.5537 - precision_6: 0.5542 - recall_6: 0.4991
Epoch 22/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6829 - acc: 0.5519 - precision_6: 0.5532 - recall_6: 0.4890 0s - loss: 0.6832 - acc: 0.5524 - precision_6: 0.552
Epoch 23/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6802 - acc: 0.5526 - precision_6: 0.5534 - recall_6: 0.4945 0s - loss: 0.6803 - acc: 0.5523 - precision_6: 0.5543 - recall_6: 0.
Epoch 24/50
257/257 [==============================] - 5s 20ms/step - loss: 0.6799 - acc: 0.5567 - precision_6: 0.5611 - recall_6: 0.4767
Epoch 25/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6793 - acc: 0.5517 - precision_6: 0.5526 - recall_6: 0.4927 1s - loss: 0.6
Epoch 26/50
257/257 [=======================

257/257 [==============================] - 3s 13ms/step - loss: 0.6845 - acc: 0.5425 - precision_7: 0.5363 - recall_7: 0.6112
Epoch 23/50
257/257 [==============================] - 3s 13ms/step - loss: 0.6846 - acc: 0.5403 - precision_7: 0.5340 - recall_7: 0.6161
Epoch 24/50
257/257 [==============================] - 3s 13ms/step - loss: 0.6844 - acc: 0.5411 - precision_7: 0.5331 - recall_7: 0.6449
Epoch 25/50
257/257 [==============================] - 3s 13ms/step - loss: 0.6836 - acc: 0.5374 - precision_7: 0.5324 - recall_7: 0.5960 1s - loss: 0.6828 - acc: 0.5
Epoch 26/50
257/257 [==============================] - 3s 13ms/step - loss: 0.6814 - acc: 0.5526 - precision_7: 0.5449 - recall_7: 0.6256
Epoch 27/50
257/257 [==============================] - 3s 13ms/step - loss: 0.6810 - acc: 0.5545 - precision_7: 0.5460 - recall_7: 0.6349
Epoch 28/50
257/257 [==============================] - 3s 13ms/step - loss: 0.6797 - acc: 0.5519 - precision_7: 0.5440 - recall_7: 0.6273
Epoch 29/50
257/2

257/257 [==============================] - 5s 21ms/step - loss: 0.6800 - acc: 0.5543 - precision_8: 0.5543 - recall_8: 0.5722 0s - loss: 0.6800 - acc: 0.5583 - precis
Epoch 21/50
257/257 [==============================] - 5s 20ms/step - loss: 0.6798 - acc: 0.5566 - precision_8: 0.5572 - recall_8: 0.5685
Epoch 22/50
257/257 [==============================] - 5s 20ms/step - loss: 0.6782 - acc: 0.5594 - precision_8: 0.5593 - recall_8: 0.5768
Epoch 23/50
257/257 [==============================] - 5s 20ms/step - loss: 0.6780 - acc: 0.5616 - precision_8: 0.5646 - recall_8: 0.5532 2s - loss: 0.6747 - acc: 0.5668  - ETA: 1s - loss:
Epoch 24/50
257/257 [==============================] - 5s 20ms/step - loss: 0.6757 - acc: 0.5622 - precision_8: 0.5651 - recall_8: 0.5549
Epoch 25/50
257/257 [==============================] - 5s 19ms/step - loss: 0.6757 - acc: 0.5681 - precision_8: 0.5684 - recall_8: 0.5794 2s - loss: 0.6717 - acc: 0.5704 - precisio - ETA: 1s - loss:
Epoch 26/50
257/257 [==========

257/257 [==============================] - 4s 17ms/step - loss: 0.6863 - acc: 0.5403 - precision_9: 0.5359 - recall_9: 0.6404
Epoch 21/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6848 - acc: 0.5427 - precision_9: 0.5373 - recall_9: 0.6530
Epoch 22/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6841 - acc: 0.5430 - precision_9: 0.5391 - recall_9: 0.6281
Epoch 23/50
257/257 [==============================] - 5s 18ms/step - loss: 0.6829 - acc: 0.5494 - precision_9: 0.5435 - recall_9: 0.6487
Epoch 24/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6825 - acc: 0.5487 - precision_9: 0.5421 - recall_9: 0.6588 1s - loss: 0.6817 - 
Epoch 25/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6822 - acc: 0.5461 - precision_9: 0.5416 - recall_9: 0.6339
Epoch 26/50
257/257 [==============================] - 4s 17ms/step - loss: 0.6818 - acc: 0.5450 - precision_9: 0.5410 - recall_9: 0.6281
Epoch 27/50
257/257 [====

In [15]:
def Average(lst):
    return sum(lst) / len(lst)

In [16]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.541199
test precision: 0.542128
test recall: 0.554267
test f1_score: 0.546263
